# Alaska Satellite Facility

Use the ASF Search API by extent and year.

Documentation is available here: https://docs.asf.alaska.edu/api/keywords/


In [3]:
import requests
import geopandas as gpd
import shapely

from utils import get_australia_coords

url = "https://api.daac.asf.alaska.edu/services/search/param?"

In [11]:
def search(start, end, coords=None):
    if coords is None:
        coords = get_australia_coords()
    coords_string = ",".join([f"{x[0]} {x[1]}" for x in coords])
    polygon = f"polygon(({coords_string}))"

    print(polygon)

    params = {
        "platform": "S1",
        "processingLevel": "GRD_HD",
        "output": "JSON",
        "start": start,
        "end": end,
        "maxResults": 12000,
        "flightDirection": "DESCENDING",
        "beamMode": "IW",
        "intersectsWith": polygon,
    }
    response = requests.get(url, params=params)
    return response.json()[0]

In [6]:
year_count = {}
for year in range(2014, 2025):
    start = f"{year}-01-01T00:00:00Z"
    end = f"{year}-12-31T23:59:59Z"
    features = search(start, end)
    year_count[year] = len(features)
    print(year, len(features))

2014 54
2015 1592
2016 3017
2017 8911
2018 9069
2019 9214
2020 8958
2021 8676
2022 5560
2023 5612
2024 1096


In [23]:
geojson = []
for feature in features:
    feature["geometry"] = shapely.from_wkt(feature["stringFootprint"])

gdf = gpd.GeoDataFrame(features).set_geometry("geometry", crs="EPSG:4326")
gdf.explore(style_kwds=dict(fillOpacity=0.05))

In [16]:
# Get tasmanian data
tas_coords = [
    [-43.7545, 144.8853],
    [-40.5736, 145.0057],
    [-40.8659, 148.5993],
    [-43.5482, 147.7396],
    [-42.6541, 148.1006],
    [-43.7545, 144.8853],
]
start = f"2023-03-01T00:00:00Z"
end = f"2023-03-31T23:59:59Z"

features = search(start, end)
features

polygon((142.61514077477727 -10.497057143968808,154.79255318148873 -26.92274564659739,147.84107484405428 -44.19519132032192,114.90782995464868 -34.86538994108997,112.14695912559668 -22.307955946507583,130.04331824962992 -10.94513221310178,142.61514077477727 -10.497057143968808))


[{'absoluteBurstID': None,
  'absoluteOrbit': '47893',
  'beamMode': 'IW',
  'beamModeType': 'IW',
  'beamSwath': None,
  'browse': 'https://datapool.asf.alaska.edu/BROWSE/SA/S1A_IW_GRDH_1SDV_20230331T205910_20230331T205936_047893_05C146_78F0.jpg',
  'catSceneId': None,
  'centerLat': '-20.4055',
  'centerLon': '127.6818',
  'collectionName': None,
  'configurationName': 'Interferometric Wide. 250 km swath, 5 m x 20 m spatial resolution and burst synchronization for interferometry. IW is considered to be the standard mode over land masses.',
  'doppler': '0',
  'downloadUrl': 'https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_IW_GRDH_1SDV_20230331T205910_20230331T205936_047893_05C146_78F0.zip',
  'farEndLat': '-20.903534',
  'farEndLon': '126.264336',
  'farStartLat': '-19.33049',
  'farStartLon': '126.683365',
  'faradayRotation': None,
  'fileName': 'S1A_IW_GRDH_1SDV_20230331T205910_20230331T205936_047893_05C146_78F0.zip',
  'finalFrame': '3992',
  'firstFrame': '3992',
  'flightDirectio

In [14]:
get_australia_coords()

[[142.61514077477727, -10.497057143968808],
 [154.79255318148873, -26.92274564659739],
 [147.84107484405428, -44.19519132032192],
 [114.90782995464868, -34.86538994108997],
 [112.14695912559668, -22.307955946507583],
 [130.04331824962992, -10.94513221310178],
 [142.61514077477727, -10.497057143968808]]